Here, I will extract hand gestures from the HAnd Gesture Recognition Image Datset (HAGRID).

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import mediapipe as mp
import numpy as np
import os

In [ ]:
mp_hands = mp.solutions.hands

hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.8)

mp_drawing = mp.solutions.drawing_utils

In [ ]:
root_dir = "D:/"

list_of_dirs = os.listdir(os.path.join(root_dir, "gestures"))

for i, dir in enumerate(list_of_dirs):
    gesture_path = os.path.join(root_dir, "gestures", dir)
    my_gesture_path = os.path.join(root_dir, "my_gestures", dir)

    if not os.path.exists(my_gesture_path):
        os.makedirs(my_gesture_path)

    print(my_gesture_path)
    print("----------")

    list_of_filenames = os.listdir(gesture_path)
    
    for j, filename in enumerate(list_of_filenames):
        
        file = os.path.join(gesture_path, filename)
        
        try:
            input_img = cv.imread(file)

            if input_img is None:
                continue
            
            img = cv.cvtColor(input_img, cv.COLOR_BGR2RGB)

            results = hands.process(img)

            if not results.multi_hand_landmarks:
                continue

            for hand_lms in results.multi_hand_landmarks:
            
                rows, cols, channels = img.shape
                    
                x_min = cols
                y_min = rows
                x_max = 0
                y_max = 0
                delta_x = 0
                delta_y = 0                   

                for n in hand_lms.landmark:
                    x = n.x * cols
                    y = n.y * rows

                    if x < x_min:
                        x_min = np.uint32(x)
                    if y < y_min:
                        y_min = np.uint32(y)

                    if x > x_max:
                        x_max = np.uint32(x)
                    if y > y_max:
                        y_max = np.uint32(y)

                delta_x = x_max - x_min
                delta_y = y_max - y_min

                multiples = 0
                for n in range(np.max(img.shape) // 28):
                    if ((n * 28) > delta_x) and ((n * 28) > delta_y):
                        multiples = n
                        break

                multiples += 2

                centre_x = x_min + delta_x // 2
                centre_y = y_min + delta_y // 2

                x_min = centre_x - (multiples * 28) // 2
                y_min = centre_y - (multiples * 28) // 2
                x_max = centre_x + (multiples * 28) // 2
                y_max = centre_y + (multiples * 28) // 2

                hand_fs = input_img[y_min:y_max, x_min:x_max, :]          
                hand_rs = cv.resize(hand_fs, (224, 224))

                cv.imwrite(os.path.join(my_gesture_path, f"{j}.jpg"), hand_rs)

        except Exception as e:
            print(e)
            continue
    
    print("Done")

Afterwards, need to go through exported images and delete wrong ones.